In [236]:
import pandas as pd
import numpy as np
import sklearn
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
import itertools

import matplotlib.gridspec as gridspec
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from mlxtend.plotting import plot_decision_regions
from sklearn.datasets import load_iris
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score, roc_curve
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [237]:
Karelia = pd.read_csv('Karelia.csv')
SP = pd.read_csv('SP.csv')

In [238]:
Karelia.drop(['name'], inplace=True, axis=1)
Karelia.drop(['feelslikemax'], inplace=True, axis=1)
Karelia.drop(['feelslikemin'], inplace=True, axis=1)
Karelia.drop(['tempmax'], inplace=True, axis=1)
Karelia.drop(['tempmin'], inplace=True, axis=1)
Karelia.drop(['dew'], inplace=True, axis=1)
Karelia.drop(['solarenergy'], inplace=True, axis=1)
Karelia.drop(['uvindex'], inplace=True, axis=1)
Karelia.drop(['severerisk'], inplace=True, axis=1)
Karelia.drop(['sunrise'], inplace=True, axis=1)
Karelia.drop(['sunset'], inplace=True, axis=1)
Karelia.drop(['moonphase'], inplace=True, axis=1)
Karelia.drop(['stations'], inplace=True, axis=1)
Karelia.drop(['description'], inplace=True, axis=1)
Karelia.drop(['precip'], inplace=True, axis=1)
Karelia.drop(['precipprob'], inplace=True, axis=1)
Karelia.drop(['precipcover'], inplace=True, axis=1)
Karelia.drop(['preciptype'], inplace=True, axis=1)
Karelia.drop(['snowdepth'], inplace=True, axis=1)
Karelia.drop(['windgust'], inplace=True, axis=1)
Karelia.drop(['winddir'], inplace=True, axis=1)
Karelia.drop(['sealevelpressure'], inplace=True, axis=1)
Karelia.drop(['cloudcover'], inplace=True, axis=1)
Karelia.drop(['visibility'], inplace=True, axis=1)
Karelia.drop(['solarradiation'], inplace=True, axis=1)
Karelia.drop(['conditions'], inplace=True, axis=1)

In [239]:
Karelia.isnull().any().any()

np.False_

In [240]:
data_year = []
data_month = []
data_day = []
for i in Karelia['datetime']:
    s = str(i).split('-')
    data_year.append(int(s[0]))
    data_month.append(int(s[1]))
    data_day.append(int(s[2]))
con = pd.DataFrame({'Year': data_year, 'Month': data_month, 'Day': data_day})
Karelia = pd.concat([Karelia, con], axis=1)
Karelia.drop(['datetime'], inplace=True, axis=1)

In [241]:
Karelia.head(3)

,temp,feelslike,humidity,snow,windspeed,icon,Year,Month,Day
0,-5.7,-11.7,82.0,0.0,21.6,snow,2022,1,1
1,-11.7,-17.2,85.7,0.0,21.6,partly-cloudy-day,2022,1,2
2,-18.4,-18.7,84.4,0.0,7.2,cloudy,2022,1,3


In [242]:
SP.drop(['name'], inplace=True, axis=1)
SP.drop(['feelslikemax'], inplace=True, axis=1)
SP.drop(['feelslikemin'], inplace=True, axis=1)
SP.drop(['tempmax'], inplace=True, axis=1)
SP.drop(['tempmin'], inplace=True, axis=1)
SP.drop(['dew'], inplace=True, axis=1)
SP.drop(['solarenergy'], inplace=True, axis=1)
SP.drop(['uvindex'], inplace=True, axis=1)
SP.drop(['severerisk'], inplace=True, axis=1)
SP.drop(['sunrise'], inplace=True, axis=1)
SP.drop(['sunset'], inplace=True, axis=1)
SP.drop(['moonphase'], inplace=True, axis=1)
SP.drop(['stations'], inplace=True, axis=1)
SP.drop(['description'], inplace=True, axis=1)
SP.drop(['precip'], inplace=True, axis=1)
SP.drop(['precipprob'], inplace=True, axis=1)
SP.drop(['precipcover'], inplace=True, axis=1)
SP.drop(['preciptype'], inplace=True, axis=1)
SP.drop(['snowdepth'], inplace=True, axis=1)
SP.drop(['windgust'], inplace=True, axis=1)
SP.drop(['winddir'], inplace=True, axis=1)
SP.drop(['sealevelpressure'], inplace=True, axis=1)
SP.drop(['cloudcover'], inplace=True, axis=1)
SP.drop(['visibility'], inplace=True, axis=1)
SP.drop(['solarradiation'], inplace=True, axis=1)
SP.drop(['conditions'], inplace=True, axis=1)

In [243]:
SP.isnull().any().any()

np.False_

In [244]:
data_year = []
data_month = []
data_day = []
for i in SP['datetime']:
    s = str(i).split('-')
    data_year.append(int(s[0]))
    data_month.append(int(s[1]))
    data_day.append(int(s[2]))
con = pd.DataFrame({'Year': data_year, 'Month': data_month, 'Day': data_day})
SP = pd.concat([SP, con], axis=1)
SP.drop(['datetime'], inplace=True, axis=1)

In [245]:
SP.head(3)

,temp,feelslike,humidity,snow,windspeed,icon,Year,Month,Day
0,-4.1,-10.7,78.9,1.9,36.0,snow,2022,1,1
1,-8.4,-13.8,85.6,0.0,36.0,partly-cloudy-day,2022,1,2
2,-6.9,-12.9,86.8,1.4,25.2,snow,2022,1,3


In [246]:
df_train = pd.read_excel('train.xlsx')
df_test = pd.read_excel('test.xlsx')

df_train

,Unnamed: 0,№ брони,Номеров,Стоимость,Внесена предоплата,Способ оплаты,Дата бронирования,Дата отмены,Заезд,Ночей,Выезд,Источник,Статус брони,Категория номера,Гостей,Гостиница
0,0,20230428-6634-194809261,1,25700.0,0,Внешняя система оплаты,2023-04-20 20:37:30,2023-04-20 20:39:15,2023-04-28 15:00:00,3,2023-05-01 12:00:00,Яндекс.Путешествия,Отмена,Номер «Стандарт»,2,1
1,1,20220711-6634-144460018,1,24800.0,12400,Отложенная электронная оплата: Банк Россия (ба...,2022-06-18 14:17:02,NaT,2022-07-11 15:00:00,2,2022-07-13 12:00:00,Официальный сайт,Активный,Номер «Стандарт»,2,1
2,2,20221204-16563-171020423,1,25800.0,12900,Банк. карта: Банк Россия (банк. карта),2022-11-14 22:59:30,NaT,2022-12-04 15:00:00,2,2022-12-06 12:00:00,Официальный сайт,Активный,Номер «Студия»,2,4
3,3,20230918-7491-223512699,1,10500.0,0,Внешняя система оплаты (С предоплатой),2023-09-08 15:55:53,NaT,2023-09-18 15:00:00,1,2023-09-19 12:00:00,Bronevik.com(new),Активный,Номер «Стандарт»,1,3
4,4,20230529-6634-200121971,1,28690.0,28690,Система быстрых платежей: Эквайринг ComfortBoo...,2023-05-20 19:54:13,NaT,2023-05-29 15:00:00,2,2023-05-31 12:00:00,Официальный сайт,Активный,Номер «Люкс»,4,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26169,26169,20230310-7492-177993190,1,18240.0,9120,Банк. карта: Банк Россия (банк. карта),2023-01-07 17:45:18,NaT,2023-03-10 15:00:00,2,2023-03-12 12:00:00,Официальный сайт,Активный,Номер «Стандарт»,2,2
26170,26170,20230625-16563-206126520,1,69600.0,23200,Банк. карта: Банк Россия (банк. карта),2023-06-20 17:54:17,NaT,2023-06-25 15:00:00,3,2023-06-28 12:00:00,Официальный сайт,Активный,Номер «Студия»,3,4
26171,26171,20220624-7492-137587082,1,55600.0,13900,Банк. карта: Банк Россия (банк. карта),2022-05-08 19:24:05,NaT,2022-06-24 15:00:00,4,2022-06-28 12:00:00,Официальный сайт,Активный,Номер «Стандарт»,2,2
26172,26172,20220427-7491-125459150,1,6300.0,0,Гарантия банковской картой,2022-02-19 09:55:50,2022-04-16 23:14:35,2022-04-27 15:00:00,1,2022-04-28 12:00:00,booking.com,Отмена,Номер «Стандарт»,2,3


In [247]:
df_train['Дата отмены'].fillna(0, inplace=True)
df_train['Дата отмены'] = np.where(df_train['Дата отмены'] != 0, 1, 0)
df_train.drop(['Unnamed: 0'], inplace=True, axis=1)
df_train.drop(['№ брони'], inplace=True, axis=1)
df_train.drop(['Выезд'], inplace=True, axis=1)
df_train.drop(['Статус брони'], inplace=True, axis=1)

#df_train.drop(['Способ оплаты'], inplace=True, axis=1)
#df_train.drop(['Источник'], inplace=True, axis=1)

C:\Users\Nikol\AppData\Local\Temp\ipykernel_23080\512713354.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_train['Дата отмены'].fillna(0, inplace=True)
C:\Users\Nikol\AppData\Local\Temp\ipykernel_23080\512713354.py:1: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0' has dtype incompatible with datetime64[ns], please explicitly cast to a compatible dtype first.
  df_train['Дата отмены'].fillna(0, inplace=True)


In [248]:
data_bron_year = []
data_bron_month = []
data_bron_day = []
data_za_year = []
data_za_month = []
data_za_day = []
for i in df_train['Дата бронирования']:
    s = str(i).split(':')[0][:-2]
    s = s.split('-')
    data_bron_year.append(int(s[0]))
    data_bron_month.append(int(s[1]))
    data_bron_day.append(int(s[2]))
for i in df_train['Заезд']:
    s = str(i).split(':')[0][:-2]
    s = s.split('-')
    data_za_year.append(int(s[0]))
    data_za_month.append(int(s[1]))
    data_za_day.append(int(s[2]))
df_con = pd.DataFrame({'Дата бронирования год': data_bron_year, 'Дата бронирования месяц': data_bron_month, 'Дата бронирования день': data_bron_day,
                       'Дата заезда год': data_za_year, 'Дата заезда месяц': data_za_month, 'Дата заезда день': data_za_day})
df_train = pd.concat([df_train, df_con], axis=1)
df_train.drop(['Дата бронирования'], inplace=True, axis=1)
df_train.drop(['Заезд'], inplace=True, axis=1)
df_train.columns

Index(['Номеров', 'Стоимость', 'Внесена предоплата', 'Способ оплаты',
       'Дата отмены', 'Ночей', 'Источник', 'Категория номера', 'Гостей',
       'Гостиница', 'Дата бронирования год', 'Дата бронирования месяц',
       'Дата бронирования день', 'Дата заезда год', 'Дата заезда месяц',
       'Дата заезда день'],
      dtype='object')

In [249]:
df_train

,Номеров,Стоимость,Внесена предоплата,Способ оплаты,Дата отмены,Ночей,Источник,Категория номера,Гостей,Гостиница,Дата бронирования год,Дата бронирования месяц,Дата бронирования день,Дата заезда год,Дата заезда месяц,Дата заезда день
0,1,25700.0,0,Внешняя система оплаты,1,3,Яндекс.Путешествия,Номер «Стандарт»,2,1,2023,4,20,2023,4,28
1,1,24800.0,12400,Отложенная электронная оплата: Банк Россия (ба...,0,2,Официальный сайт,Номер «Стандарт»,2,1,2022,6,18,2022,7,11
2,1,25800.0,12900,Банк. карта: Банк Россия (банк. карта),0,2,Официальный сайт,Номер «Студия»,2,4,2022,11,14,2022,12,4
3,1,10500.0,0,Внешняя система оплаты (С предоплатой),0,1,Bronevik.com(new),Номер «Стандарт»,1,3,2023,9,8,2023,9,18
4,1,28690.0,28690,Система быстрых платежей: Эквайринг ComfortBoo...,0,2,Официальный сайт,Номер «Люкс»,4,1,2023,5,20,2023,5,29
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26169,1,18240.0,9120,Банк. карта: Банк Россия (банк. карта),0,2,Официальный сайт,Номер «Стандарт»,2,2,2023,1,7,2023,3,10
26170,1,69600.0,23200,Банк. карта: Банк Россия (банк. карта),0,3,Официальный сайт,Номер «Студия»,3,4,2023,6,20,2023,6,25
26171,1,55600.0,13900,Банк. карта: Банк Россия (банк. карта),0,4,Официальный сайт,Номер «Стандарт»,2,2,2022,5,8,2022,6,24
26172,1,6300.0,0,Гарантия банковской картой,1,1,booking.com,Номер «Стандарт»,2,3,2022,2,19,2022,4,27


In [250]:
tmp = []
feelslike = []
humidity = []
snow = []
windspeed = []
icon = []

In [251]:
for i in range(len(df_train)):
    if df_train['Гостиница'].iloc[i] > 2:
        tmp.append(Karelia[(Karelia['Year'] == df_train['Дата заезда год'].iloc[i]) & (Karelia['Month'] == df_train['Дата заезда месяц'].iloc[i]) & (Karelia['Day'] ==  df_train['Дата заезда день'].iloc[i])]['temp'].iloc[0])
        feelslike.append(Karelia[(Karelia['Year'] == df_train['Дата заезда год'].iloc[i]) & (Karelia['Month'] == df_train['Дата заезда месяц'].iloc[i]) & (Karelia['Day'] ==  df_train['Дата заезда день'].iloc[i])]['feelslike'].iloc[0])
        humidity.append(Karelia[(Karelia['Year'] == df_train['Дата заезда год'].iloc[i]) & (Karelia['Month'] == df_train['Дата заезда месяц'].iloc[i]) & (Karelia['Day'] ==  df_train['Дата заезда день'].iloc[i])]['humidity'].iloc[0])
        snow.append(Karelia[(Karelia['Year'] == df_train['Дата заезда год'].iloc[i]) & (Karelia['Month'] == df_train['Дата заезда месяц'].iloc[i]) & (Karelia['Day'] ==  df_train['Дата заезда день'].iloc[i])]['snow'].iloc[0])
        windspeed.append(Karelia[(Karelia['Year'] == df_train['Дата заезда год'].iloc[i]) & (Karelia['Month'] == df_train['Дата заезда месяц'].iloc[i]) & (Karelia['Day'] ==  df_train['Дата заезда день'].iloc[i])]['windspeed'].iloc[0])
        icon.append(Karelia[(Karelia['Year'] == df_train['Дата заезда год'].iloc[i]) & (Karelia['Month'] == df_train['Дата заезда месяц'].iloc[i]) & (Karelia['Day'] ==  df_train['Дата заезда день'].iloc[i])]['icon'].iloc[0])
    else:
        tmp.append(SP[(SP['Year'] == df_train['Дата заезда год'].iloc[i]) & (SP['Month'] == df_train['Дата заезда месяц'].iloc[i]) & (SP['Day'] ==  df_train['Дата заезда день'].iloc[i])]['temp'].iloc[0])
        feelslike.append(SP[(SP['Year'] == df_train['Дата заезда год'].iloc[i]) & (SP['Month'] == df_train['Дата заезда месяц'].iloc[i]) & (SP['Day'] ==  df_train['Дата заезда день'].iloc[i])]['feelslike'].iloc[0])
        humidity.append(SP[(SP['Year'] == df_train['Дата заезда год'].iloc[i]) & (SP['Month'] == df_train['Дата заезда месяц'].iloc[i]) & (SP['Day'] ==  df_train['Дата заезда день'].iloc[i])]['humidity'].iloc[0])
        snow.append(SP[(SP['Year'] == df_train['Дата заезда год'].iloc[i]) & (SP['Month'] == df_train['Дата заезда месяц'].iloc[i]) & (SP['Day'] ==  df_train['Дата заезда день'].iloc[i])]['snow'].iloc[0])
        windspeed.append(SP[(SP['Year'] == df_train['Дата заезда год'].iloc[i]) & (SP['Month'] == df_train['Дата заезда месяц'].iloc[i]) & (SP['Day'] ==  df_train['Дата заезда день'].iloc[i])]['windspeed'].iloc[0])
        icon.append(SP[(SP['Year'] == df_train['Дата заезда год'].iloc[i]) & (SP['Month'] == df_train['Дата заезда месяц'].iloc[i]) & (SP['Day'] ==  df_train['Дата заезда день'].iloc[i])]['icon'].iloc[0])

In [252]:
df_train['temprature']= tmp
df_train['feelslike'] = feelslike
df_train['humidity'] = humidity
df_train['snow'] = snow
df_train['windspeed'] = windspeed
df_train['icon'] = icon

In [253]:
dic = {'Номер «Стандарт»':[],
      'Номер «Студия»':[],
      'Номер «Люкс»':[],
      'Апартаменты с 2 спальнями с отдельным входом':[],
      'Коттедж с 3 спальнями':[],
      'Коттедж с 2 спальнями':[],
      'Номер «Стандарт» для маломобильных групп населения':[]}
count = 0
for i in df_train['Категория номера']:
    s = i.split('. ')
    if len(s) == 1:
        dic[s[0]].append(1)
        for j in dic.keys():
            if j != s[0]:
                dic[j].append(0)
    else:
        app = list(dic.keys())
        for k in s[1::]:
            temp = k
            if k[-1].isnumeric():
                temp = k[:-2]
            
            try:
                app.remove(temp)
            except ValueError:
                pass
            
            try:
                dic[temp][count] += 1
            except IndexError:
                dic[temp].append(1)

        for t in app:
            dic[t].append(0)
    count += 1

df_train = pd.concat([df_train, pd.DataFrame(dic)], axis=1)
df_train.drop(['Категория номера'], inplace=True, axis=1)    

In [254]:
df_train

,Номеров,Стоимость,Внесена предоплата,Способ оплаты,Дата отмены,Ночей,Источник,Гостей,Гостиница,Дата бронирования год,...,snow,windspeed,icon,Номер «Стандарт»,Номер «Студия»,Номер «Люкс»,Апартаменты с 2 спальнями с отдельным входом,Коттедж с 3 спальнями,Коттедж с 2 спальнями,Номер «Стандарт» для маломобильных групп населения
0,1,25700.0,0,Внешняя система оплаты,1,3,Яндекс.Путешествия,2,1,2023,...,0.0,25.2,rain,1,0,0,0,0,0,0
1,1,24800.0,12400,Отложенная электронная оплата: Банк Россия (ба...,0,2,Официальный сайт,2,1,2022,...,0.0,21.6,rain,1,0,0,0,0,0,0
2,1,25800.0,12900,Банк. карта: Банк Россия (банк. карта),0,2,Официальный сайт,2,4,2022,...,6.3,14.4,snow,0,1,0,0,0,0,0
3,1,10500.0,0,Внешняя система оплаты (С предоплатой),0,1,Bronevik.com(new),1,3,2023,...,0.0,25.2,rain,1,0,0,0,0,0,0
4,1,28690.0,28690,Система быстрых платежей: Эквайринг ComfortBoo...,0,2,Официальный сайт,4,1,2023,...,0.0,28.8,rain,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26169,1,18240.0,9120,Банк. карта: Банк Россия (банк. карта),0,2,Официальный сайт,2,2,2023,...,0.0,18.0,partly-cloudy-day,1,0,0,0,0,0,0
26170,1,69600.0,23200,Банк. карта: Банк Россия (банк. карта),0,3,Официальный сайт,3,4,2023,...,0.0,14.4,partly-cloudy-day,0,1,0,0,0,0,0
26171,1,55600.0,13900,Банк. карта: Банк Россия (банк. карта),0,4,Официальный сайт,2,2,2022,...,0.0,14.4,partly-cloudy-day,1,0,0,0,0,0,0
26172,1,6300.0,0,Гарантия банковской картой,1,1,booking.com,2,3,2022,...,0.2,25.2,snow,1,0,0,0,0,0,0


In [255]:
df_train_dm = pd.get_dummies(data=df_train, columns=['Способ оплаты', 'Источник', 'icon'])

In [256]:
df_train_dm = df_train_dm.reindex(columns=sorted(list(df_train_dm.columns)))

In [257]:
import re
regex = re.compile(r"\[|\]|<", re.IGNORECASE)
df_train_dm.columns = [regex.sub("_", col) if any(x in str(col) for x in set(('[', ']', '<'))) else col for col in df_train_dm.columns.values]

In [258]:
X = df_train_dm.drop(['Дата отмены'], axis=1)
y = df_train_dm['Дата отмены']

In [259]:
X_for_train = X.copy()
y_for_train = y.copy()

In [260]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report
from sklearn.svm import SVC

X_train, X_test, y_train, y_test= train_test_split(X, y, test_size=0.33, random_state=1, shuffle=True)

In [62]:
max_depth = [2, 4, 7, 10, 15, 20, 30, 50]
min_samples_leaf = [2, 5, 10, 20, 35, 50, 70, 85, 100]
rf = DecisionTreeClassifier()
par = {'max_depth': max_depth, 'min_samples_leaf': min_samples_leaf}
gs = GridSearchCV(estimator=rf, param_grid=par, scoring='roc_auc', n_jobs=-1, cv=5, error_score='raise')
gs.fit(X_train, y_train)

C:\Users\Nikol\AppData\Local\Programs\Python\Python312\Lib\site-packages\numpy\ma\core.py:2881: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,


GridSearchCV(cv=5, error_score='raise', estimator=DecisionTreeClassifier(),
             n_jobs=-1,
             param_grid={'max_depth': [2, 4, 7, 10, 15, 20, 30, 50],
                         'min_samples_leaf': [2, 5, 10, 20, 35, 50, 70, 85,
                                              100]},
             scoring='roc_auc')

In [63]:
gs.best_params_

{'max_depth': 7, 'min_samples_leaf': 2}

In [64]:
gs.best_score_

np.float64(0.8172881200125571)

In [68]:
lr = LogisticRegression(max_iter=100)
#svm = SVC(kernel="linear", max_iter=10)

lr.fit(X_train, y_train)
#svm.fit(X_train, y_train)

lr_pred = lr.predict(X_test)
#svm_pred = svm.predict(X_test)

print(f'auc_roc {roc_auc_score(y_test, lr_pred)}')
#print(f'auc_roc {roc_auc_score(y_test, svm_pred)}')

auc_roc 0.6401363001878755


C:\Users\Nikol\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [66]:
# rf = DecisionTreeClassifier(max_depth = 300, min_samples_split= 35, n_estimators= 300)
rf = DecisionTreeClassifier(**gs.best_params_)
rf.fit(X_train, y_train)
pred = rf.predict(X_test)
print(f'auc_roc {roc_auc_score(y_test, pred)}')

auc_roc 0.7316014619523981


In [195]:
from xgboost.sklearn import XGBRegressor
from sklearn.model_selection import GridSearchCV

params = {
    'max_depth' : [2, 3, 5, 10, 15, 25],
    'learning_rate' : [0.01, 0.1, 0.15, 0.25, 0.4, 0.5],
    'n_estimators' : [1, 10, 30, 50, 100],
}

model_xgb = XGBRegressor(random_state=76)

gs = GridSearchCV(
    estimator=model_xgb, param_grid=params, scoring='neg_root_mean_squared_error', n_jobs=-1, cv=5, error_score='raise'
)

In [261]:
gs.fit(X_train, y_train)

C:\Users\Nikol\AppData\Local\Programs\Python\Python312\Lib\site-packages\numpy\ma\core.py:2881: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,


GridSearchCV(cv=5, error_score='raise',
             estimator=XGBRegressor(base_score=None, booster=None,
                                    callbacks=None, colsample_bylevel=None,
                                    colsample_bynode=None,
                                    colsample_bytree=None, device=None,
                                    early_stopping_rounds=None,
                                    enable_categorical=False, eval_metric=None,
                                    feature_types=None, gamma=None,
                                    grow_policy=None, importance_type=None,
                                    interaction_constraints=None,...
                                    max_depth=None, max_leaves=None,
                                    min_child_weight=None, missing=nan,
                                    monotone_constraints=None,
                                    multi_strategy=None, n_estimators=None,
                                    n_jobs=None, num_parallel_tree=None,
                                    random_state=76, ...),
             n_jobs=-1,
             param_grid={'learning_rate': [0.01, 0.1, 0.15, 0.25, 0.4, 0.5],
                         'max_depth': [2, 3, 5, 10, 15, 25],
                         'n_estimators': [1, 10, 30, 50, 100]},
             scoring='neg_root_mean_squared_error')

In [262]:
gs.best_params_, gs.best_score_

({'learning_rate': 0.1, 'max_depth': 5, 'n_estimators': 100},
 np.float64(-0.2912662537464298))

In [263]:
model_xgb = XGBRegressor(**gs.best_params_, random_state=76)

model_xgb.fit(X_train, y_train)

y_pred_train = model_xgb.predict(X_train)
y_pred_test = model_xgb.predict(X_test)

In [264]:
print(f'auc_roc {roc_auc_score(y_train, y_pred_train)}')
print(f'auc_roc {roc_auc_score(y_test, y_pred_test)}')

auc_roc 0.9089454355445254
auc_roc 0.8236895984241137


ТРЕЙН

In [234]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report
from sklearn.svm import SVC

In [237]:
from xgboost.sklearn import XGBRegressor
from sklearn.model_selection import GridSearchCV

params = {
    'max_depth' : [2, 3, 5, 10, 15, 25],
    'learning_rate' : [0.01, 0.1, 0.15, 0.25, 0.4, 0.5],
    'n_estimators' : [1, 10, 30, 50, 100],
}

model_xgb = XGBRegressor(random_state=76)

gs_ = GridSearchCV(
    estimator=model_xgb, param_grid=params, scoring='neg_root_mean_squared_error', n_jobs=-1, cv=5, error_score='raise'
)

In [ ]:
gs_.fit(X_for_train, y_for_train)
gs_.best_params_, gs_.best_score_

In [ ]:
model_xgb_ = XGBRegressor(**gs_.best_params_, random_state=76)

model_xgb_.fit(X_for_train, y_for_train)

y_pred = model_xgb_.predict(X_for_train)

In [ ]:
print(f'auc_roc {roc_auc_score(y_train, y_pred)}')

# ТЕСТ

In [265]:
df_test = pd.read_excel('test.xlsx')
df_test

,Unnamed: 0,№ брони,Номеров,Стоимость,Внесена предоплата,Способ оплаты,Дата бронирования,Заезд,Ночей,Выезд,Источник,Категория номера,Гостей,Гостиница
0,0,20231129-16563-238946689,1,23750.0,23750,Банк. карта: Банк Россия (банк. карта),2023-11-28 10:34:43,2023-11-29 15:00:00,2,2023-12-01 12:00:00,Официальный сайт,Номер «Студия»,3,4
1,1,20221219-7491-174959103,1,15010.0,7505,Банк. карта: Банк Россия (банк. карта),2022-12-12 18:30:43,2022-12-19 15:00:00,2,2022-12-21 12:00:00,Официальный сайт,Номер «Стандарт»,2,3
2,2,20221211-6634-172724329,1,8400.0,8400,Отложенная электронная оплата: Банк Россия (ба...,2022-11-25 22:03:59,2022-12-11 15:00:00,1,2022-12-12 12:00:00,Официальный сайт,Номер «Стандарт»,2,1
3,3,20230821-6634-212247350,1,42500.0,42500,Банк. карта: Банк Россия (банк. карта),2023-07-18 15:45:46,2023-08-21 15:00:00,3,2023-08-24 12:00:00,Официальный сайт,Апартаменты с 2 спальнями с отдельным входом,4,1
4,4,20230326-6634-189784563,1,62500.0,11900,Система быстрых платежей: Эквайринг ComfortBoo...,2023-03-23 11:04:13,2023-03-26 15:00:00,5,2023-03-31 12:00:00,Официальный сайт,Номер «Стандарт»,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11213,11213,20220507-7492-130458541,1,50200.0,50200,Банк. карта [Кешбэк. МИР]: Эквайринг TravelLin...,2022-03-27 21:30:38,2022-05-07 15:00:00,2,2022-05-09 12:00:00,Официальный сайт,Номер «Студия»,4,2
11214,11214,20240217-6634-235901857,1,190100.0,43500,Система быстрых платежей: Эквайринг ComfortBoo...,2023-11-11 17:51:55,2024-02-16 15:00:00,5,2024-02-21 12:00:00,Официальный сайт,Коттедж с 2 спальнями,4,1
11215,11215,20220209-6634-124107676,1,42300.0,42300,Банк. карта: Банк Россия (банк. карта),2022-02-09 02:23:14,2022-02-09 15:00:00,1,2022-02-10 12:00:00,Официальный сайт,Коттедж с 3 спальнями,5,1
11216,11216,20230128-6634-179977236,1,27900.0,27900,Банк. карта: Банк Россия (банк. карта),2023-01-21 09:10:17,2023-01-28 15:00:00,1,2023-01-29 12:00:00,Официальный сайт,Апартаменты с 2 спальнями с отдельным входом,4,1


In [266]:
df_test.drop(['Unnamed: 0'], inplace=True, axis=1)
df_test.drop(['№ брони'], inplace=True, axis=1)
df_test.drop(['Выезд'], inplace=True, axis=1)
data_bron_year = []
data_bron_month = []
data_bron_day = []
data_za_year = []
data_za_month = []
data_za_day = []
for i in df_test['Дата бронирования']:
    s = str(i).split(':')[0][:-2]
    s = s.split('-')
    data_bron_year.append(int(s[0]))
    data_bron_month.append(int(s[1]))
    data_bron_day.append(int(s[2]))
for i in df_test['Заезд']:
    s = str(i).split(':')[0][:-2]
    s = s.split('-')
    data_za_year.append(int(s[0]))
    data_za_month.append(int(s[1]))
    data_za_day.append(int(s[2]))
df_con_ts = pd.DataFrame({'Дата бронирования год': data_bron_year, 'Дата бронирования месяц': data_bron_month, 'Дата бронирования день': data_bron_day,
                       'Дата заезда год': data_za_year, 'Дата заезда месяц': data_za_month, 'Дата заезда день': data_za_day})
df_test = pd.concat([df_test, df_con_ts], axis=1)
df_test.drop(['Дата бронирования'], inplace=True, axis=1)
df_test.drop(['Заезд'], inplace=True, axis=1)

tmp = []
feelslike = []
humidity = []
snow = []
windspeed = []
icon = []
for i in range(len(df_test)):
    if df_test['Гостиница'].iloc[i] > 2:
        tmp.append(Karelia[(Karelia['Year'] == df_test['Дата заезда год'].iloc[i]) & (Karelia['Month'] == df_test['Дата заезда месяц'].iloc[i]) & (Karelia['Day'] ==  df_test['Дата заезда день'].iloc[i])]['temp'].iloc[0])
        feelslike.append(Karelia[(Karelia['Year'] == df_test['Дата заезда год'].iloc[i]) & (Karelia['Month'] == df_test['Дата заезда месяц'].iloc[i]) & (Karelia['Day'] ==  df_test['Дата заезда день'].iloc[i])]['feelslike'].iloc[0])
        humidity.append(Karelia[(Karelia['Year'] == df_test['Дата заезда год'].iloc[i]) & (Karelia['Month'] == df_test['Дата заезда месяц'].iloc[i]) & (Karelia['Day'] ==  df_test['Дата заезда день'].iloc[i])]['humidity'].iloc[0])
        snow.append(Karelia[(Karelia['Year'] == df_test['Дата заезда год'].iloc[i]) & (Karelia['Month'] == df_test['Дата заезда месяц'].iloc[i]) & (Karelia['Day'] ==  df_test['Дата заезда день'].iloc[i])]['snow'].iloc[0])
        windspeed.append(Karelia[(Karelia['Year'] == df_test['Дата заезда год'].iloc[i]) & (Karelia['Month'] == df_test['Дата заезда месяц'].iloc[i]) & (Karelia['Day'] ==  df_test['Дата заезда день'].iloc[i])]['windspeed'].iloc[0])
        icon.append(Karelia[(Karelia['Year'] == df_test['Дата заезда год'].iloc[i]) & (Karelia['Month'] == df_test['Дата заезда месяц'].iloc[i]) & (Karelia['Day'] ==  df_test['Дата заезда день'].iloc[i])]['icon'].iloc[0])
    else:
        tmp.append(SP[(SP['Year'] == df_test['Дата заезда год'].iloc[i]) & (SP['Month'] == df_test['Дата заезда месяц'].iloc[i]) & (SP['Day'] ==  df_test['Дата заезда день'].iloc[i])]['temp'].iloc[0])
        feelslike.append(SP[(SP['Year'] == df_test['Дата заезда год'].iloc[i]) & (SP['Month'] == df_test['Дата заезда месяц'].iloc[i]) & (SP['Day'] ==  df_test['Дата заезда день'].iloc[i])]['feelslike'].iloc[0])
        humidity.append(SP[(SP['Year'] == df_test['Дата заезда год'].iloc[i]) & (SP['Month'] == df_test['Дата заезда месяц'].iloc[i]) & (SP['Day'] ==  df_test['Дата заезда день'].iloc[i])]['humidity'].iloc[0])
        snow.append(SP[(SP['Year'] == df_test['Дата заезда год'].iloc[i]) & (SP['Month'] == df_test['Дата заезда месяц'].iloc[i]) & (SP['Day'] ==  df_test['Дата заезда день'].iloc[i])]['snow'].iloc[0])
        windspeed.append(SP[(SP['Year'] == df_test['Дата заезда год'].iloc[i]) & (SP['Month'] == df_test['Дата заезда месяц'].iloc[i]) & (SP['Day'] ==  df_test['Дата заезда день'].iloc[i])]['windspeed'].iloc[0])
        icon.append(SP[(SP['Year'] == df_test['Дата заезда год'].iloc[i]) & (SP['Month'] == df_test['Дата заезда месяц'].iloc[i]) & (SP['Day'] ==  df_test['Дата заезда день'].iloc[i])]['icon'].iloc[0])
df_test['temprature']= tmp
df_test['feelslike'] = feelslike
df_test['humidity'] = humidity
df_test['snow'] = snow
df_test['windspeed'] = windspeed
df_test['icon'] = icon

dic = {'Номер «Стандарт»':[],
      'Номер «Студия»':[],
      'Номер «Люкс»':[],
      'Апартаменты с 2 спальнями с отдельным входом':[],
      'Коттедж с 3 спальнями':[],
      'Коттедж с 2 спальнями':[],
      'Номер «Стандарт» для маломобильных групп населения':[]}
count = 0
for i in df_test['Категория номера']:
    s = i.split('. ')
    if len(s) == 1:
        dic[s[0]].append(1)
        for j in dic.keys():
            if j != s[0]:
                dic[j].append(0)
    else:
        app = list(dic.keys())
        for k in s[1::]:
            temp = k
            if k[-1].isnumeric():
                temp = k[:-2]
            
            try:
                app.remove(temp)
            except ValueError:
                pass
            
            try:
                dic[temp][count] += 1
            except IndexError:
                dic[temp].append(1)

        for t in app:
            dic[t].append(0)
    count += 1

df_test = pd.concat([df_test, pd.DataFrame(dic)], axis=1)
df_test.drop(['Категория номера'], inplace=True, axis=1) 

In [267]:
df_test_dm = pd.get_dummies(data=df_test, columns=['Способ оплаты', 'Источник', 'icon'])
df_test_dm = pd.concat([df_test_dm, pd.DataFrame({'Источник_Alean.ru (31.10.2023-22.06.2025)': [False] * df_test_dm.shape[0], 
                                                'Источник_Ozon': [False] * df_test_dm.shape[0],
                                                'Источник_expedia.com (A-Hotels.com)': [False] * df_test_dm.shape[0],
                                                'Источник_Тинькофф Путешествия': [False] * df_test_dm.shape[0]})], axis=1)

In [268]:
df_test_dm = df_test_dm.reindex(columns=sorted(list(df_test_dm.columns)))

In [269]:
import re
regex = re.compile(r"\[|\]|<", re.IGNORECASE)
df_test_dm.columns = [regex.sub("_", col) if any(x in str(col) for x in set(('[', ']', '<'))) else col for col in df_test_dm.columns.values]

In [270]:
df_test_dm

,feelslike,humidity,icon_clear-day,icon_cloudy,icon_partly-cloudy-day,icon_rain,icon_snow,snow,temprature,windspeed,...,Способ оплаты_Банк. карта: Эквайринг ComfortBooking (Банк. карта),Способ оплаты_Внешняя система оплаты,Способ оплаты_Внешняя система оплаты (Банковская карта),Способ оплаты_Внешняя система оплаты (Оплата наличными),Способ оплаты_Внешняя система оплаты (С предоплатой),Способ оплаты_Гарантия банковской картой,Способ оплаты_Отложенная электронная оплата: Банк Россия (банк. карта),Способ оплаты_При заселении,Способ оплаты_Система быстрых платежей: Эквайринг ComfortBooking (Система быстрых платежей),Стоимость
0,-5.3,89.9,False,False,False,False,True,2.1,-3.7,10.8,...,False,False,False,False,False,False,False,False,False,23750.0
1,-8.6,87.3,False,False,False,False,True,0.5,-5.1,18.0,...,False,False,False,False,False,False,False,False,False,15010.0
2,-10.9,77.8,False,True,False,False,False,1.2,-5.7,32.4,...,False,False,False,False,False,False,True,False,False,8400.0
3,16.8,69.5,False,False,True,False,False,0.0,16.9,14.4,...,False,False,False,False,False,False,False,False,False,42500.0
4,0.2,93.3,False,False,False,False,True,0.5,2.3,18.0,...,False,False,False,False,False,False,False,False,True,62500.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11213,12.7,56.9,False,False,False,True,False,0.0,13.0,28.8,...,False,False,False,False,False,False,False,False,False,50200.0
11214,-7.3,85.6,False,False,False,False,True,4.3,-2.4,28.8,...,False,False,False,False,False,False,False,False,True,190100.0
11215,-4.4,88.6,False,False,False,False,True,0.7,-0.5,21.6,...,False,False,False,False,False,False,False,False,False,42300.0
11216,-9.7,82.6,False,False,False,False,True,0.3,-4.5,28.8,...,False,False,False,False,False,False,False,False,False,27900.0


In [272]:
pred = model_xgb.predict(df_test_dm)
print(pred)

np.savetxt("test.csv", pred, delimiter="\n")

[0.06561404 0.09369186 0.06500275 ... 0.03782092 0.05592133 0.1930559 ]


In [200]:
cnt = 0
for i in pred:
    cnt += i

print(cnt)

1558
